# Borrador - Proyecto Final: Implementación de un Sistema RAG (Retrieval-Augmented Generation)

## Ejemplo - Análisis de un Sistema RAG

**Instalación de la biblioteca Transformers:**

Esta celda instala la biblioteca `transformers`, desarrollada por Hugging Face,
que proporciona modelos de procesamiento de lenguaje natural (NLP) preentrenados.

```python
!pip install transformers


In [ ]:
!pip install transformers

**Instalación de bibliotecas para NLP y recuperación de información:**

Esta celda instala varias bibliotecas esenciales para el procesamiento de lenguaje natural (NLP) y la búsqueda eficiente de vectores:

- `transformers`: Modelos preentrenados de Hugging Face para tareas de NLP.
- `faiss-cpu`: Biblioteca de Facebook AI para búsqueda eficiente de vectores de alta dimensión.
- `sentence-transformers`: Implementaciones optimizadas de modelos de embeddings de oraciones.
- `datasets`: Conjunto de datos preprocesados para entrenamiento y evaluación de modelos de NLP.

```python
!pip install transformers faiss-cpu sentence-transformers datasets


In [ ]:
!pip install transformers faiss-cpu sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


**Paso 1: Preparar el dataset**

Esta celda crea un conjunto de datos en un `DataFrame` de Pandas con información sobre diversos temas.  
Los datos se almacenan en un archivo CSV llamado `knowledge_base.csv` sin incluir el índice.

### Columnas del CSV:
- **title**: Título del tema.
- **content**: Descripción breve del tema.

Crea una base de conocimiento estructurada en un archivo `JSON` o `CSV`. Ejemplo:

In [ ]:
import pandas as pd

data = [
    {"title": "Python Basics", "content": "Python is a versatile programming language."},
    {"title": "Google Colab", "content": "Google Colab is a free platform for Python coding."},
    {"title": "RAG", "content": "RAG combines retrieval with generative models for context-based answers."}
]
df = pd.DataFrame(data)
df.to_csv("knowledge_base.csv", index=False)


**Paso 2: Índice de recuperación con embeddings**

Se carga una base de conocimientos desde un archivo CSV, genera embeddings para el contenido  
de los documentos utilizando `SentenceTransformer`, y crea un índice FAISS para búsqueda eficiente.

### Pasos:
1. **Cargar el dataset**: Se lee el archivo `knowledge_base.csv` en un `DataFrame` de Pandas.
2. **Generar embeddings**: Se usa el modelo `all-MiniLM-L6-v2` para convertir los textos en vectores.
3. **Crear el índice FAISS**: Se utiliza `IndexFlatL2` para almacenar y buscar los embeddings.
4. **Agregar los embeddings al índice**.
5. **Imprimir el número de documentos indexados**.

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

# Cargar dataset
df = pd.read_csv("knowledge_base.csv")

# Crear embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo rápido y eficiente
embeddings = model.encode(df['content'].tolist())

# Crear índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print(f"Documentos indexados: {index.ntotal}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Documentos indexados: 3


**Paso 3: Consulta al índice**

Busca los documentos más relevantes para una consulta.

In [ ]:
def search(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [df.iloc[idx]['content'] for idx in indices[0]]
    return results

query = "¿Qué es RAG?"
top_docs = search(query)
print("Documentos relevantes:")
for doc in top_docs:
    print("-", doc)


Documentos relevantes:
- RAG combines retrieval with generative models for context-based answers.
- Python is a versatile programming language.
- Google Colab is a free platform for Python coding.


**Paso 4: Generación con un modelo preentrenado**

En esta celda se implementa un flujo para combinar recuperación de información con generación de texto,  
utilizando la biblioteca `transformers` y un modelo `Flan-T5` de Google.

### Pasos:
1. **Cargar el modelo generativo**:
   - Se utiliza el pipeline `text2text-generation` con el modelo `google/flan-t5-small`.
2. **Definir la función `generate_answer`**:
   - Recupera documentos relacionados con la consulta utilizando la función `search`.
   - Combina los documentos recuperados en un contexto.
   - Crea un prompt para generar una respuesta contextualizada.
   - Devuelve la respuesta generada por el modelo.

> **Nota**: Se requiere implementar previamente la función `search` para la recuperación de documentos.


In [ ]:
from transformers import pipeline

# Cargar modelo generativo
generator = pipeline('text2text-generation', model='google/flan-t5-small')

# Combinar recuperación y generación
def generate_answer(query):
    retrieved_docs = search(query)
    context = " ".join(retrieved_docs)
    prompt = f"Contexto: {context}\nPregunta: {query}\nRespuesta:"
    result = generator(prompt, max_length=100, do_sample=True)
    return result[0]['generated_text']

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Se ejecuta la función `generate_answer` con la consulta **"¿Qué es Google Colab?"**  
para obtener una respuesta generada por el modelo.

### Pasos:
1. **Definir la consulta (`query`)**.
2. **Llamar a `generate_answer(query)`** para generar una respuesta basada en documentos recuperados.
3. **Imprimir la respuesta generada**.

In [ ]:
query = "¿Qué es Google Colab?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: free platform for Python


Dos ejecuciones pueden producir respuestas diferentes debido a la aleatoriedad en la generación.


In [ ]:
query = "¿Qué es Google Colab?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: Google Colab


## Ejemplo usando data de prueba

**Librerías necesarias:**

Primero, instala las librerías necesarias para la extracción de texto y procesamiento.

In [ ]:
!pip install PyPDF2 sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


**Extraer texto de los PDFs**

Se recorre una carpeta con archivos PDF, extrae el texto de cada uno y lo almacena  
en una lista de diccionarios con el nombre del archivo y su contenido.

Pasos:
1. **Definir la ruta de la carpeta**: Se especifica el directorio que contiene los archivos PDF.
2. **Función `extract_text_from_pdf`**: Extrae el texto de cada página de un archivo PDF utilizando `PyPDF2`.
3. **Procesar todos los archivos PDF**: Se recorre la carpeta, procesando solo los archivos con extensión `.pdf`, y se guarda el nombre del archivo junto con su contenido extraído.


In [ ]:
import os
from PyPDF2 import PdfReader

# Ruta a la carpeta con los PDFs
pdf_folder = "./data/"

# Extraer texto de los PDFs
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Procesar los PDFs en la carpeta
pdf_data = []
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        pdf_data.append({"file_name": filename, "content": extract_text_from_pdf(filepath)})

In [ ]:
# Crear dataframe
df_pdfs = pd.DataFrame(pdf_data)
print(df_pdfs.head())  # Ver las primeras filas

                 file_name                                            content
0  Plan_Luisa_Gonzalez.pdf   \n \n \n \n \n \n \n \n  \n \nPLAN DE TRABAJO...
1    Plan_Daniel_Noboa.pdf    \n1 \n \n  \n \n \n \n \n \n \n \n \n \nPLAN...


**Preprocesar los textos**

define una función para dividir un texto en fragmentos de un tamaño máximo definido,  
utilizando como referencia la cantidad de palabras en cada fragmento.

### Detalles:
- **Entrada**: Un string (`text`) que se desea dividir.
- **Parámetro `chunk_size`**: Número máximo de palabras por fragmento.  
  Valor por defecto: 300.
- **Proceso**:
  1. Se utiliza una expresión regular para dividir el texto en oraciones o líneas, usando puntos (`.`) o saltos de línea (`\n`).
  2. Se acumulan oraciones en un fragmento hasta que se alcanza el tamaño máximo especificado.
  3. Se guardan los fragmentos generados en una lista.
- **Salida**: Una lista de strings, donde cada string representa un fragmento del texto original.


In [ ]:
import re

# Función para dividir el texto en fragmentos
def split_into_chunks(text, chunk_size=300):
    sentences = re.split(r'\.|\n', text)  # Dividir en oraciones o líneas
    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        if current_size + len(sentence.split()) > chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_size = 0
        current_chunk.append(sentence)
        current_size += len(sentence.split())

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

Al aplicar la función `split_into_chunks` a cada documento PDF procesado previamente,  
dividiendo el contenido de cada PDF en fragmentos y luego almacenando estos fragmentos en un nuevo DataFrame.

Pasos:
1. **Aplicar la función de fragmentación**: Se utiliza la función `split_into_chunks` para dividir el contenido de cada PDF en fragmentos de tamaño controlado.
2. **Guardar los fragmentos**: Cada fragmento generado se almacena junto con el nombre del archivo de origen en una lista `chunk_data`.
3. **Crear el DataFrame**: Se crea un nuevo DataFrame `df_chunks` con los fragmentos, donde cada fila contiene el nombre del archivo y un fragmento de texto.
4. **Ver los primeros fragmentos**: Se imprime una vista de las primeras filas del DataFrame para verificar los resultados.


In [ ]:
# Aplicar la división en fragmentos
chunk_data = []
for _, row in df_pdfs.iterrows():
    chunks = split_into_chunks(row["content"])
    for chunk in chunks:
        chunk_data.append({"file_name": row["file_name"], "content": chunk})

# Crear dataframe de fragmentos
df_chunks = pd.DataFrame(chunk_data)
print(df_chunks.head())  # Ver las primeras filas

                 file_name                                            content
0  Plan_Luisa_Gonzalez.pdf  PLAN DE TRABAJO DEL BINOMIO   PRESIDENCIAL DE ...
1  Plan_Luisa_Gonzalez.pdf  51 Lucha contra la discriminación y violencia ...
2  Plan_Luisa_Gonzalez.pdf  Por eso nos hemos convocado contra la violenci...
3  Plan_Luisa_Gonzalez.pdf  Nos negamos a dejar que nuestros sueños se des...
4  Plan_Luisa_Gonzalez.pdf  Justicia digital y nueva economía de la inform...


**Crear el índice con embeddings**

genera embeddings para cada fragmento de texto en el DataFrame `df_chunks`,  
luego construye un índice FAISS para permitir la búsqueda eficiente de los fragmentos más similares.

Pasos:
1. **Generar embeddings**: Utiliza el modelo `SentenceTransformer` para crear representaciones vectoriales (embeddings) de cada fragmento de texto en el DataFrame `df_chunks`.
2. **Construir el índice FAISS**: Se utiliza `faiss.IndexFlatL2` para crear un índice basado en los embeddings, que permite realizar búsquedas rápidas de similitud.
3. **Agregar embeddings al DataFrame**: Los embeddings generados se agregan como una nueva columna al DataFrame `df_chunks`.
4. **Ver los resultados**: Se imprime una vista de las primeras filas del DataFrame para verificar que los embeddings se han añadido correctamente.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Crear embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df_chunks['content'].tolist())

# Construir índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Agregar embeddings al dataframe
df_chunks['embedding'] = list(embeddings)
print(df_chunks.head())  # Visualizar fragmentos con embeddings

                 file_name                                            content  \
0  Plan_Luisa_Gonzalez.pdf  PLAN DE TRABAJO DEL BINOMIO   PRESIDENCIAL DE ...   
1  Plan_Luisa_Gonzalez.pdf  51 Lucha contra la discriminación y violencia ...   
2  Plan_Luisa_Gonzalez.pdf  Por eso nos hemos convocado contra la violenci...   
3  Plan_Luisa_Gonzalez.pdf  Nos negamos a dejar que nuestros sueños se des...   
4  Plan_Luisa_Gonzalez.pdf  Justicia digital y nueva economía de la inform...   

                                           embedding  
0  [-0.03643843, -0.026629005, 0.00657822, -0.043...  
1  [0.04816845, 0.05876354, -0.092447616, -0.0066...  
2  [-0.007650564, 0.061900992, -0.08514175, -0.10...  
3  [-0.004212706, 0.032797024, -0.02077905, -0.07...  
4  [0.02621746, -0.005422394, -0.04560686, -0.100...  


**Consultar al índice**

Ahora, se define una función `search` para realizar consultas sobre el índice FAISS y recuperar los fragmentos más relevantes  
en función de la similitud de los embeddings.

Pasos:
1. **Generar el embedding de la consulta**: La consulta se convierte en un vector utilizando el modelo `SentenceTransformer`.
2. **Buscar en el índice FAISS**: Se realiza una búsqueda utilizando el índice FAISS para encontrar los fragmentos más cercanos al embedding de la consulta.
3. **Recuperar los fragmentos**: Los fragmentos más relevantes se recuperan del DataFrame `df_chunks`, junto con las distancias (similitudes) con respecto a la consulta.
4. **Visualizar los resultados**: Se muestran los resultados, incluyendo el nombre del archivo, el contenido del fragmento y la distancia de similitud.


In [ ]:
# Buscar en el índice
def search(query, top_k=10):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = df_chunks.iloc[indices[0]].copy()  # Recuperar fragmentos relevantes
    results['distance'] = distances[0]          # Agregar las distancias
    return results

# Ejemplo de consulta
query = "¿Cuál es la posición de Daniel Noboa con respecto a la violencia?"
results = search(query)

# Visualizar resultados
print(results[['file_name', 'content', 'distance']])

                  file_name  \
3   Plan_Luisa_Gonzalez.pdf   
2   Plan_Luisa_Gonzalez.pdf   
85    Plan_Daniel_Noboa.pdf   
18  Plan_Luisa_Gonzalez.pdf   
56  Plan_Luisa_Gonzalez.pdf   
40  Plan_Luisa_Gonzalez.pdf   
86    Plan_Daniel_Noboa.pdf   
89    Plan_Daniel_Noboa.pdf   
91    Plan_Daniel_Noboa.pdf   
77    Plan_Daniel_Noboa.pdf   

                                              content  distance  
3   Nos negamos a dejar que nuestros sueños se des...  0.932014  
2   Por eso nos hemos convocado contra la violenci...  0.975228  
85  La pobreza, la desigualdad, la falta de oportu...  1.041729  
18  inseguridad, como la pobreza, la exclusión soc...  1.067311  
56  mutuo entre todas las naciones Rechazamos cual...  1.073811  
40  violencia doméstica, la violencia sexual, el a...  1.074381  
86  educación en valores, el respeto a los demás, ...  1.078895  
89  sus vidas, tanto en el espacio físico como en ...  1.093904  
91  la sobrepoblación en las prisiones y permitir ...  1.104724 

**Generar respuestas con el modelo generativo**

Usamos Flan-T5 para crear respuestas basadas en el contexto recuperado. Se combina la recuperación de fragmentos relevantes con un modelo generativo para responder preguntas de manera clara y precisa.

Pasos:

1. **Cargar el modelo generativo**: Se utiliza el modelo `google/flan-t5-small` para generar respuestas a partir de un contexto.
2. **Función de truncado**: La función `truncate_context` se encarga de recortar el contexto a un máximo de 512 tokens, para cumplir con el límite del modelo de generación.
3. **Generación de respuesta**: La función `generate_answer` recupera los fragmentos más relevantes para la consulta, genera un contexto y lo pasa al modelo generativo para obtener una respuesta estructurada.
4. **Ejemplo de consulta**: Se realiza un ejemplo de consulta sobre la posición de Daniel Noboa con respecto al empleo.


In [ ]:
from transformers import pipeline

# Cargar modelo generativo
generator = pipeline('text2text-generation', model='google/flan-t5-small', device=-1)  # Usar CPU si no hay GPU

# Función para recortar el contexto a un máximo de 512 tokens
def truncate_context(context, max_tokens=512):
    tokens = context.split()  # Dividir el contexto en palabras
    truncated = " ".join(tokens[:max_tokens])  # Recortar al límite permitido
    return truncated

# Combinar recuperación y generación
def generate_answer(query, top_k=3):
    # Recuperar fragmentos relevantes
    retrieved_docs = search(query, top_k=top_k)
    context = " ".join(retrieved_docs['content'].tolist())

    # Truncar contexto al límite del modelo
    truncated_context = truncate_context(context)

    # Crear el prompt estructurado
    prompt = (
        f"Contexto: {truncated_context}\n"
        f"Pregunta: {query}\n"
        f"Por favor, responde de manera clara y precisa comenzando con:\n"
        f"\"La posición de {query.split()[4]} con respecto al empleo es ...\".\n"
        f"Respuesta:"
    )

    # Generar la respuesta
    result = generator(prompt, max_length=200, do_sample=True)

    # Devolver la respuesta generada
    return result[0]['generated_text']

# Ejemplo de generación
query = "¿Cuál es la posición de Daniel Noboa con respecto al empleo?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1425 > 512). Running this sequence through the model will result in indexing errors


Respuesta generada: Para volver a ser Patria! En este marco, nuestro Objetivo General and Objetivos Prioritados, se describen a continuación: OBJETIVO GENERAL : Alcanzar el buen vivir en una democracia justa igualaria, con un Estado plurinacional e intercultural de derecho y justicia, que promueva nuestra  libertades, capacidades y aspiraciones en una sociedad solidaria y cuiales oportunidades económicas, polticas, culturales y ecológicas, para que todos ecuálticas, con iléctricas oportunidades,


**Personalización de preguntas**

Se genera una respuesta utilizando el modelo FLAN-T5, recuperando los fragmentos relevantes y generando una respuesta a la consulta dada.

Ejemplo de consulta:

Se realiza una consulta sobre la posición de Luisa González respecto al empleo.

In [ ]:
query = "¿Cuál es la posición de Luisa González respecto al empleo?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: In referencia a toma de trabajo en Ecuador, por lo que tomar medidas radicales peroda, participación de colectivos, sociedad civil, pueblos y nonacionalidades son de una sueo más libre y determinante en que enviamos las proximidades de la participación colectivo en el fortud de la democracia y en la toma de decisiones colectivos, que en seguridad equilibrada y definición, en la que la actividad que producen sueo estupéntica una forma de participación incluirse incluirse con mucha la sociedad ecatólica Estado pl
